# Optimizing Model Parameters

Tutorial link: https://docs.pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%
100.0%
100.0%
100.0%


In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300932  [   64/60000]
loss: 2.293195  [ 6464/60000]
loss: 2.270026  [12864/60000]
loss: 2.268075  [19264/60000]
loss: 2.250946  [25664/60000]
loss: 2.221660  [32064/60000]
loss: 2.230323  [38464/60000]
loss: 2.202998  [44864/60000]
loss: 2.202245  [51264/60000]
loss: 2.169335  [57664/60000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 2.163961 

Epoch 2
-------------------------------
loss: 2.170569  [   64/60000]
loss: 2.163696  [ 6464/60000]
loss: 2.106141  [12864/60000]
loss: 2.122396  [19264/60000]
loss: 2.070552  [25664/60000]
loss: 2.020853  [32064/60000]
loss: 2.040438  [38464/60000]
loss: 1.972732  [44864/60000]
loss: 1.976068  [51264/60000]
loss: 1.910450  [57664/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.904378 

Epoch 3
-------------------------------
loss: 1.928076  [   64/60000]
loss: 1.906367  [ 6464/60000]
loss: 1.790166  [12864/60000]
loss: 1.825132  [19264/60000]
loss: 1.716079  [25664/60000]
loss: 1.675971  [32064/600